# Can we get language models to accurately extract the rationale from all the Oversight Board's decisions?

In [2]:
from buttermilk import BM
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

bm = BM()
print(bm.cfg)

# v = compose(config_name="config", )
# abs_config_dir=os.getcwd() + "/conf"
# with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
#     cfg = compose(config_name="config")


{'name': 'default_project', 'job': 'development', 'secret_provider': 'azure', 'logger': 'gcp', 'verbose': True, 'models_secret': 'models', 'concurrent': 10, 'gcp': {'project': 'dmrc-platforms', 'region': 'us-central1', 'bucket': 'dmrc-analysis'}, 'azure': {'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 'rg-suzor_ai'}, 'aws': {'secret_name': 'secret', 'region': 'us-east-1'}, 'save': {'type': 'bq', 'destination': 'dmrc-analysis.toxicity.flow', 'schema': 'buttermilk/schemas/flow.json'}, 'data': {'runs': {'type': 'job', 'name': 'runs', 'dataset': 'dmrc-analysis.toxicity.flow', 'last_n_days': 7, 'aggregate': False, 'max_records_per_group': 1, 'filter': None, 'group': {'job_id': 'job_id', 'record_id': 'record.record_id'}, 'columns': {'timestamp': 'timestamp', 'agent': 'agent_info.agent', 'source': 'source', 'error': 'error', 'parameters': 'parameters'}}}}


In [ ]:
from buttermilk.data.loaders import LoaderGCS
from buttermilk.agents.agent import Agent
from buttermilk._core.runner_types import RecordInfo
from buttermilk.runner.orchestrator import MultiFlowOrchestrator

async def case_generator():
    reader = LoaderGCS(glob="gs://dmrc-platforms/data/osb/*.md")

    async for uri, case in reader.read_files(1):
        yield uri, RecordInfo(content=case)

agent_vars = dict(template='summarise_osb', template_vars={},
                  save_params=bm.cfg.save, concurrent=bm.cfg.concurrent)
run_vars = dict(model=['gpt4o', 'gemini15pro', 'sonnet','llama3_70bn'])

orchestrator = MultiFlowOrchestrator(n_runs=1, agent_vars=agent_vars, run_vars=run_vars, job_generator=)


